### Must run step by step as there are command lines to be executed in the middle of the flow

### Input files: Price-Today.csv
### Output files: daily-sales.csv, daily-sales-prices.csv

In [3]:
import numpy as np
import os
import pandas as pd
from datetime import date, timedelta, datetime
from sqlalchemy import create_engine
from pandas.tseries.offsets import BDay

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

today = date.today()
print(today)

2025-02-06


In [5]:
# convert the timedelta object to a BusinessDay object
num_business_days = BDay(1)
yesterday = today - num_business_days
yesterday = yesterday.date()
print(f'Today: {today}')
print(f'Yesterday: {yesterday}')

Today: 2025-02-06
Yesterday: 2025-02-05


In [7]:
cols = 'name fm_date to_date fm_price to_price qty max_price min_price percent status'.split()

format_dict = {
    'fm_price':'{:.2f}','to_price':'{:.2f}','diff':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'volume':'{:,.2f}','beta':'{:,.2f}',
    'pct':'{:,.2f}%','percent':'{:,.2f}%',   
    'fm_date':'{:%Y-%m-%d}','to_date':'{:%Y-%m-%d}',
    'created_at':'{:%Y-%m-%d}','updated_at':'{:%Y-%m-%d}',
    
    'qty':'{:,}','available_qty':'{:,}',
    'cost':'{:.2f}','buy_target':'{:.2f}','sell_target':'{:.2f}',
}

In [9]:
# Get the user's home directory
user_path = os.path.expanduser('~')
# Get the current working directory
current_path = os.getcwd()
# Derive the base directory (base_dir) by removing the last folder ('Daily')
base_path = os.path.dirname(current_path)
#C:\Users\PC1\OneDrive\A5\Data
dat_path = os.path.join(base_path, "Data")
#C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data>
god_path = os.path.join(user_path, "OneDrive","Imports","santisoontarinka@gmail.com - Google Drive","Data")
#C:\Users\PC1\iCloudDrive\data
icd_path = os.path.join(user_path, "iCloudDrive", "Data")
#C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data
osd_path = os.path.join(user_path, "OneDrive","Documents","obsidian-git-sync","Data")

In [11]:
print("User path:", user_path)
print(f"Current path: {current_path}")
print(f"Base path: {base_path}")
print(f"Data path : {dat_path}") 
print(f"Google Drive path : {god_path}")
print(f"iCloudDrive path : {icd_path}") 
print(f"OSD path : {osd_path}") 

User path: C:\Users\PC1
Current path: C:\Users\PC1\OneDrive\A5\Daily
Base path: C:\Users\PC1\OneDrive\A5
Data path : C:\Users\PC1\OneDrive\A5\Data
Google Drive path : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data
iCloudDrive path : C:\Users\PC1\iCloudDrive\Data
OSD path : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data


In [13]:
sql = """
SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '%s'
ORDER BY a.name
"""
sql = sql % yesterday
print(sql)
tmp = pd.read_sql(sql, conlite)
tmp.shape


SELECT DISTINCT a.name
FROM sales a 
WHERE to_date = '2025-02-05'
ORDER BY a.name



(20, 1)

In [15]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % yesterday
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2025-02-05' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



### This statement causes program to hang when there is no data

In [18]:
df = pd.read_sql(sql, conlite)
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
#df.style.format(format_dict)
df

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,3BBIF,2025-01-22,2025-02-05,5.15,5.45,443060840,5.45,5.10,B,SET,0.30,5.83
1,AIMIRT,2025-01-31,2025-02-05,10.50,10.50,1467280,10.50,10.40,B,SET,0.00,0.00
2,CPNREIT,2025-01-31,2025-02-05,12.10,12.00,8848400,12.40,12.00,B,SET,-0.10,-0.83
3,DIF,2025-01-30,2025-02-05,8.40,8.45,143665320,8.55,8.35,B,SET,0.05,0.60
4,GVREIT,2025-02-03,2025-02-05,6.75,6.60,7419956,6.75,6.50,B,SET,-0.15,-2.22
5,JMART,2025-01-30,2025-02-05,11.30,10.40,73091144,12.00,10.40,B,SET50,-0.90,-7.96
6,KCE,2025-02-05,2025-02-05,21.00,20.10,58047096,23.20,20.10,B,SET100,-0.90,-4.29
7,MCS,2025-01-29,2025-02-05,6.25,6.00,2152408,6.30,6.00,B,SET,-0.25,-4.00
8,NER,2025-01-29,2025-02-05,4.98,4.82,111144344,5.05,4.82,B,SET,-0.16,-3.21
9,SENA,2025-01-29,2025-02-05,2.18,2.16,10313920,2.20,2.16,B,SET,-0.02,-0.92


In [20]:
df.shape

(20, 12)

### IF the above count not equal number of orders, there must be something incorrect

### Create daily-sales from sales

In [24]:
df[cols].style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status
0,3BBIF,2025-01-22,2025-02-05,5.15,5.45,"443,060,840",5.45,5.10,5.83%,B
1,AIMIRT,2025-01-31,2025-02-05,10.50,10.50,"1,467,280",10.50,10.40,0.00%,B
2,CPNREIT,2025-01-31,2025-02-05,12.10,12.00,"8,848,400",12.40,12.00,-0.83%,B
3,DIF,2025-01-30,2025-02-05,8.40,8.45,"143,665,320",8.55,8.35,0.60%,B
4,GVREIT,2025-02-03,2025-02-05,6.75,6.60,"7,419,956",6.75,6.50,-2.22%,B
5,JMART,2025-01-30,2025-02-05,11.30,10.40,"73,091,144",12.00,10.40,-7.96%,B
6,KCE,2025-02-05,2025-02-05,21.00,20.10,"58,047,096",23.20,20.10,-4.29%,B
7,MCS,2025-01-29,2025-02-05,6.25,6.00,"2,152,408",6.30,6.00,-4.00%,B
8,NER,2025-01-29,2025-02-05,4.98,4.82,"111,144,344",5.05,4.82,-3.21%,B
9,SENA,2025-01-29,2025-02-05,2.18,2.16,"10,313,920",2.20,2.16,-0.92%,B


In [26]:
file_name = "daily-sales.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [28]:
print(f"Output file : {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file : C:\Users\PC1\OneDrive\A5\Data\daily-sales.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\daily-sales.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\daily-sales.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\daily-sales.csv


In [30]:
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(god_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(icd_file, header=True, index=False)
df[cols].sort_values(['status','percent'],ascending=[True,True]).to_csv(osd_file, header=True, index=False)

In [32]:
sales = df[cols]
sales.shape

(20, 10)

In [34]:
file_name = "Price-Today.csv"
input_file = os.path.join(dat_path, file_name)
print(f"Input file: {input_file}")

Input file: C:\Users\PC1\OneDrive\A5\Data\Price-Today.csv


In [36]:
prices = pd.read_csv(input_file)
prices.shape

(182, 10)

In [38]:
df_merge = pd.merge(sales,prices,on='name', how='inner')
df_merge.shape

(20, 19)

In [40]:
colu = 'name fm_date to_date fm_price to_price qty max_price min_price percent status \
price change volume date'.split()
df_merge[colu]

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,percent,status,price,change,volume,date
0,3BBIF,2025-01-22,2025-02-05,5.15,5.45,443060840,5.45,5.10,5.83,B,5.40,-0.05,12848463,2025-02-06
1,AIMIRT,2025-01-31,2025-02-05,10.50,10.50,1467280,10.50,10.40,0.00,B,10.40,-0.10,87050,2025-02-06
2,CPNREIT,2025-01-31,2025-02-05,12.10,12.00,8848400,12.40,12.00,-0.83,B,12.00,0.00,1412265,2025-02-06
3,DIF,2025-01-30,2025-02-05,8.40,8.45,143665320,8.55,8.35,0.60,B,8.45,0.00,6460027,2025-02-06
4,GVREIT,2025-02-03,2025-02-05,6.75,6.60,7419956,6.75,6.50,-2.22,B,6.60,0.00,124310,2025-02-06
5,JMART,2025-01-30,2025-02-05,11.30,10.40,73091144,12.00,10.40,-7.96,B,9.95,-0.45,10586139,2025-02-06
6,KCE,2025-02-05,2025-02-05,21.00,20.10,58047096,23.20,20.10,-4.29,B,19.70,-0.40,6495328,2025-02-06
7,MCS,2025-01-29,2025-02-05,6.25,6.00,2152408,6.30,6.00,-4.00,B,5.95,-0.05,149895,2025-02-06
8,NER,2025-01-29,2025-02-05,4.98,4.82,111144344,5.05,4.82,-3.21,B,4.68,-0.14,7247897,2025-02-06
9,SENA,2025-01-29,2025-02-05,2.18,2.16,10313920,2.20,2.16,-0.92,B,2.16,0.00,267138,2025-02-06


In [42]:
file_name = "daily-sales-prices.csv"
output_file = os.path.join(dat_path, file_name)
god_file = os.path.join(god_path, file_name)
icd_file = os.path.join(icd_path, file_name)
osd_file = os.path.join(osd_path, file_name)

In [44]:
print(f"Output file: {output_file}") 
print(f"icd_file : {icd_file}") 
print(f"god_file : {god_file}") 
print(f"osd_file : {osd_file}") 

Output file: C:\Users\PC1\OneDrive\A5\Data\daily-sales-prices.csv
icd_file : C:\Users\PC1\iCloudDrive\Data\daily-sales-prices.csv
god_file : C:\Users\PC1\OneDrive\Imports\santisoontarinka@gmail.com - Google Drive\Data\daily-sales-prices.csv
osd_file : C:\Users\PC1\OneDrive\Documents\obsidian-git-sync\Data\daily-sales-prices.csv


In [46]:
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(output_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(god_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(icd_file, header=True, index=False)
df_merge[colu].sort_values(['name'],ascending=[True]).to_csv(osd_file, header=True, index=False)

### Add or update sells record depends on trend

In [49]:
os.chdir(base_path)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5'

In [51]:
!ruby ruby\\daily-out-new.rb

Name      From Date    To Date   From     To     Pct      Shares    Max    Min S Action
---------------------------------------------------------------------------------------
3BBIF    2025-02-06 2025-02-06   5.45   5.40  -0.92% 443,060,840   5.45   5.10 B Insert
AIMIRT   2025-01-31 2025-02-06  10.50  10.40  -0.95%   1,467,280  10.50  10.40 B Update
AWC      2025-01-29 2025-02-06   3.18   2.92  -8.18% 647,108,144   3.24   2.92 S Update
BCH      2025-01-30 2025-02-06  14.30  13.70   -4.2%  71,942,040  14.60  13.70 S Update
CPNREIT  2025-01-31 2025-02-06  12.10  12.00  -0.83%   8,848,400  12.40  12.00 B Update
DIF      2025-01-30 2025-02-06   8.40   8.45    0.6% 143,665,320   8.55   8.35 B Update
GVREIT   2025-02-03 2025-02-06   6.75   6.60  -2.22%   7,419,956   6.75   6.50 B Update
JMART    2025-01-30 2025-02-06  11.30   9.95 -11.95%  73,091,144  12.00   9.95 B Update
JMT      2025-01-30 2025-02-06  15.30  14.00   -8.5% 162,452,872  15.70  14.00 S Update
KCE      2025-02-05 2025-02-06  

In [53]:
os.chdir(current_path)
%pwd

'C:\\Users\\PC1\\OneDrive\\A5\\Daily'



### cd\ruby\port_lite


### rails runner db/ins_sales.rb



In [55]:
sql = """
SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '%s' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name
"""
sql = sql % today
print(sql)


SELECT a.name,fm_date,to_date,fm_price,to_price,
a.qty,a.max_price,a.min_price,t.status,t.market
FROM sales a 
JOIN stocks t ON a.name = t.name 
WHERE to_date = '2025-02-06' AND t.status IN ("B","I", "O", "S") 
ORDER BY t.status, a.name



In [57]:
df = pd.read_sql(sql, conlite)
df.shape

(20, 10)

In [59]:
#df.set_index(["name"], inplace=True)
df['fm_date'] = pd.to_datetime(df['fm_date'])
df['to_date'] = pd.to_datetime(df['to_date'])
df.eval('diff = to_price - fm_price',inplace=True)
df['percent'] = round(df['diff']/df['fm_price']*100,2)
df.style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
0,3BBIF,2025-02-06,2025-02-06,5.45,5.40,"443,060,840",5.45,5.10,B,SET,-0.05,-0.92%
1,AIMIRT,2025-01-31,2025-02-06,10.50,10.40,"2,934,560",10.50,10.40,B,SET,-0.10,-0.95%
2,CPNREIT,2025-01-31,2025-02-06,12.10,12.00,"17,696,800",12.40,12.00,B,SET,-0.10,-0.83%
3,DIF,2025-01-30,2025-02-06,8.40,8.45,"287,330,640",8.55,8.35,B,SET,0.05,0.60%
4,GVREIT,2025-02-03,2025-02-06,6.75,6.60,"14,839,912",6.75,6.50,B,SET,-0.15,-2.22%
5,JMART,2025-01-30,2025-02-06,11.30,9.95,"146,182,288",12.00,9.95,B,SET50,-1.35,-11.95%
6,KCE,2025-02-05,2025-02-06,21.00,19.70,"116,094,192",23.20,19.70,B,SET100,-1.30,-6.19%
7,MCS,2025-01-29,2025-02-06,6.25,5.95,"4,304,816",6.30,5.95,B,SET,-0.30,-4.80%
8,NER,2025-01-29,2025-02-06,4.98,4.68,"222,288,688",5.05,4.68,B,SET,-0.30,-6.02%
9,SENA,2025-01-29,2025-02-06,2.18,2.16,"20,627,840",2.20,2.16,B,SET,-0.02,-0.92%


In [61]:
df_sort = df.sort_values(by=['percent'],ascending=[True]).copy()

In [63]:
df_sort.query('percent <= -5.00').style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent
17,SYNEX,2025-01-31,2025-02-06,13.40,11.20,"9,912,496",13.40,11.20,S,SET,-2.20,-16.42%
5,JMART,2025-01-30,2025-02-06,11.30,9.95,"146,182,288",12.00,9.95,B,SET50,-1.35,-11.95%
15,PTG,2025-01-29,2025-02-06,7.70,6.85,"72,359,568",7.80,6.85,S,SET100,-0.85,-11.04%
16,RCL,2025-01-27,2025-02-06,24.90,22.60,"144,296,224",25.50,22.60,S,SET100,-2.30,-9.24%
14,JMT,2025-01-30,2025-02-06,15.30,14.00,"324,905,744",15.70,14.00,S,SET50,-1.30,-8.50%
12,AWC,2025-01-29,2025-02-06,3.18,2.92,"1,294,216,288",3.24,2.92,S,SET50,-0.26,-8.18%
6,KCE,2025-02-05,2025-02-06,21.00,19.70,"116,094,192",23.20,19.70,B,SET100,-1.30,-6.19%
8,NER,2025-01-29,2025-02-06,4.98,4.68,"222,288,688",5.05,4.68,B,SET,-0.30,-6.02%


In [65]:
df_sort.query('percent >= 5.00').style.format(format_dict)

,name,fm_date,to_date,fm_price,to_price,qty,max_price,min_price,status,market,diff,percent


In [67]:
conlite.close()

In [69]:
# Get the current time
current_time = datetime.now()
# Format the time to remove milliseconds
formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
print(formatted_time)

2025-02-06 20:43:18
